# 測自動抓取3頁搜尋結果內容(title 到 dtail ),

# 所有funtion 合在一起

In [3]:
# 觀察搜尋結果網址,用網址切換頁數
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
import csv
s = Service("./chromedriver.exe")
driver = webdriver.Chrome(service=s)
driver.get("https://www.yes123.com.tw/wk_index/joblist.asp?search_job_t=1&strrec=0&search_key_word=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB&search_type=job&search_item=1&search_from=index")
allkey = driver.page_source
time.sleep(2) 
soup = BeautifulSoup(str(allkey), 'html.parser')
page=soup.find("div",{"id":"select_page_num"}).find_all("option")[:1]
yes123_host = "https://www.yes123.com.tw/wk_index/"
pages=[]
for i in page:    
    allpage_value=int(i.text)
    pages.append(allpage_value)
pages.insert(0,0)
search_links=[]
for search_pagevalue in pages :
    search_link_part="joblist.asp?search_job_t=1&strrec="
    search_link_part2= "&search_key_word=後端工程師&search_from=joblist"
    search_link = yes123_host+search_link_part+str(search_pagevalue*30)+search_link_part2
    search_links.append(search_link)
search_pagesources=[]
for search_pagesource in search_links:
    driver.get(search_pagesource)
    time.sleep(2)
    page_source = driver.page_source
    page_source_soup = BeautifulSoup(str(page_source), 'html.parser')
    search_pagesources.append(page_source_soup)
link_results=[]
results2=[]
for search_pagesource in  search_pagesources :
    job_titlepage_div = search_pagesource.find_all("div",{"class":"Job_opening_item"})
    for link in job_titlepage_div:
        try:
            all_part_links =link.find("h5").find("a").get("href")
            all_title_links = yes123_host+all_part_links
            print(all_title_links)    
            link_results.append(all_title_links)
            all_titles = link.find("h5").get_text()
            all_bus= link.find("h6").get_text()
            all_addr=link.find("div",{"class":"Job_opening_item_info"}).find("span").text
            print(all_addr)
            result2 ={
                     '職缺': all_titles,
                    '公司名稱': all_bus,
                    '公司地址': all_addr
                }
            results2.append(result2)
        except:
            continue
detail_pagesource_results=[] 
print(len(results2))
for all_detail_page in link_results:

    driver.get(all_detail_page)
    time.sleep(2)
    onepage_detail_pagesource = driver.page_source
    time.sleep(1) 
    soup2 = BeautifulSoup(str(onepage_detail_pagesource), 'html.parser')   
    detail_pagesource_results.append(soup2)     
driver.close()
all_h3_list =[]
detail_results=[]
for new in detail_pagesource_results:         
    test2 = new.find_all("div",{"class":"job_explain"})[1:] 
    h3_results=[]
    for okok in test2:
        h3_result=str(okok.find("h3"))   
        h3_results.append(h3_result)   
    try:
        a=h3_results.index("<h3>工作條件</h3>")
        # print(a)
        test4 = new.find_all("div",{"class":"job_explain"})[a+1].find_all("span",{"class":"right_main"})[0].text
        test3 = new.find_all("div",{"class":"job_explain"})[a+1].find_all("span",{"class":"right_main"})[1].text        
    except:
        test3=" "
        test4=" "
    job_salary_div = new.find("div",{"class":"job_explain mt"}).find_all("span",{"class":"right_main"})[1].text  
    job_content_div = new.find("div",{"class":"job_explain mt"}).find_all("span",{"class":"right_main"})[0].text  
    try:
        b=h3_results.index("<h3>技能與求職專長</h3>")
        job_can_div = new.find_all("div",{"class":"job_explain"})[b+1].find_all("span",{"class":"right_main"})
        for i1 in job_can_div:
            job_can_text =i1.text          
    except:
        job_can_text=" "   
    try:
        c=h3_results.index("<h3>其他條件</h3>")
        job_other_div = new.find_all("div",{"class":"job_explain"})[c+1].find_all("span",{"class":"exception"})
        for i2 in job_other_div:
            job_other_text=i2.text           
    except:
        job_other_text=" "      
    detail_result ={
             '學歷': test4,
            '科系': test3,
            '薪資':  job_salary_div,
            '工作內容':job_content_div,
            '擅長工具':job_can_text,
            '其他條件':job_other_text
        }
    detail_results.append(detail_result)
for i in range(min(len(detail_results),len(results2))):
    detail_results[i].update(results2[i])
labels =  ['職缺','公司名稱','公司地址','學歷','科系','薪資','工作內容','擅長工具','其他條件']
dct_arr = detail_results
try:
    with open('03page_all_column.csv', 'w',encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=labels)
        writer.writeheader()
        for elem in dct_arr:
            writer.writerow(elem)
except IOError:
     print("I/O error")

https://www.yes123.com.tw/wk_index/job.asp?p_id=20151214172036_24955602&job_id=20220721171146_67820800
新北市新莊區 
https://www.yes123.com.tw/wk_index/job.asp?p_id=20220311120149_82871877&job_id=20220311134057_72709037
台北市南港區 
https://www.yes123.com.tw/wk_index/job.asp?p_id=20210504114231_54194279&job_id=20210504140029_290901
高雄市三民區 
https://www.yes123.com.tw/wk_index/job.asp?p_id=20151214172036_24955602&job_id=20220721174108_89695205
新北市新莊區 
https://www.yes123.com.tw/wk_index/job.asp?p_id=9119793_12868358&job_id=20220307095733_4825070
新北市汐止區 
https://www.yes123.com.tw/wk_index/job.asp?p_id=9119793_12868358&job_id=20220418035324_55903567
新北市汐止區 
https://www.yes123.com.tw/wk_index/job.asp?p_id=9119793_12868358&job_id=20220627035836_6194484
新北市汐止區 
https://www.yes123.com.tw/wk_index/job.asp?p_id=6480938_86380802&job_id=20220322032020_6051237
台北市內湖區 
https://www.yes123.com.tw/wk_index/job.asp?p_id=20110211160133_28771443&job_id=20140124095456_13304731
台北市大安區 
https://www.yes123.com.tw/wk_index

# 局部測試

In [27]:
print(len(pages))

5


In [ ]:
print(pages[-1])
search_links=[]
for search_pagevalue in pages :
    print(search_pagevalue)
    search_link_part="joblist.asp?search_job_t=1&strrec="
    search_link_part2= "&search_key_word=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB&search_type=job&search_item=1&search_from=index"
    search_link = search_link_part+str(search_pagevalue*30)+search_link_part2
    print(search_link)
    search_links.append(search_link)
search_pagesources=[]
for search_pagesource in search_links:
    driver.get(search_pagesource)
    time.sleep(2)
    page_source = driver.page_source
    search_pagesources.append(page_source)
print(len(search_pagesources))
    

    
    

In [1]:
# detail的完整連結
search_link_part="joblist.asp?search_job_t=1&strrec="
search_link_part2= "&search_key_word=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB&search_type=job&search_item=1&search_from=index"
search_link = search_link_part+search_pagevalue+search_link_part2


NameError: name 'search_pagevalue' is not defined

In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
import csv
s = Service("./chromedriver.exe")
driver = webdriver.Chrome(service=s)
driver.get("https://www.yes123.com.tw/wk_index/joblist.asp?search_job_t=1&strrec=0&search_key_word=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB&search_type=job&search_item=1&search_from=index")
allkey = driver.page_source
time.sleep(2)
soup = BeautifulSoup(str(allkey), 'html.parser')
yes123_host = "https://www.yes123.com.tw/wk_index/"
page=soup.find("div",{"id":"select_page_num"}).find_all("option")[:4]
job_titlepage_div = soup.find_all("div",{"class":"Job_opening_item"})
pages=[]
for i in page:    
    allpage_value=int(i.text)
    pages.append(allpage_value)
    # print(allpage_value)

pages.insert(0,0)
print(pages)

results2=[]  
link_results=[]
page_source_results = []
search_all_pagesource = []
for pagevalue in pages:
    onepage_detail_pagesource = driver.page_source
    time.sleep(1)    
    soup2 = BeautifulSoup(str(onepage_detail_pagesource), 'html.parser')
    search_all_pagesource.append(soup2)       
    s4=Select(driver.find_element(By.ID,'inputState'))  
    time.sleep(1)
    s4.select_by_index(pagevalue) 
    time.sleep(10)

[0, 1, 2, 3, 4]


In [39]:
# print(search_all_pagesource[3])
w= search_all_pagesource[2].find_all("div",{"class":"Job_opening_item"})
print(w)

[<div class="Job_opening_item">
<div class="Job_opening_item_date">
<div class="d-flex">
<div class="manufacturing_icon"></div>
<div>06.20</div>
</div>
</div>
<div class="d-flex">
<div class="Job_opening_item_title">
<h5><a href="job.asp?p_id=9119793_12868358&amp;job_id=20220425035753_212399" target="_blank">Python後端/架構師_高雄_21017</a></h5>
<h6><a href="comp_info.asp?p_id=9119793_12868358" target="_blank">緯創資通股份有限公司</a></h6>
<div class="Job_opening_item_title_payment">
<div class="icon_commonStyle payment_icon"></div>
<span>薪資面議(經常性薪資達4萬元含以上)</span>
</div>
</div>
<div class="Job_opening_item_info">
<div>
<div>
<div class="icon_commonStyle location_icon"></div>
</div>
<span>高雄市苓雅區 </span>
</div>
<div>
<div>
<div class="icon_commonStyle user_icon"></div>
</div>
<span>全職</span>
</div>
<div class="hashTag d-flex flex-wrap">
</div>
</div>
</div>
</div>, <div class="Job_opening_item">
<div class="Job_opening_item_date">
<div class="d-flex">
<div class="exclusive_icon"></div>
<div>06.02</div>
<

In [ ]:
all_h3_list =[]
detail_results=[]
for new in detail_pagesource_results:         
    test2 = new.find_all("div",{"class":"job_explain"})[1:-1] 
    h3_results=[]
    for okok in test2:
        h3_result=okok.find("h3").text
        h3_results.append(h3_result)
    # print("(學歷,科系要求)=============================================================")
    try:
        a=h3_results.index("工作條件")
        # print(a)
        test4 = new.find_all("div",{"class":"job_explain"})[a+1].find_all("span",{"class":"right_main"})[0].text
        test3 = new.find_all("div",{"class":"job_explain"})[a+1].find_all("span",{"class":"right_main"})[1].text
        # print(test4)
        # print(test3)      
    except:
        test3=" "
        test4=" "
        # print(test4)
        # print(test3)
    # print("(薪資)=============================================================")
    job_salary_div = new.find("div",{"class":"job_explain mt"}).find_all("span",{"class":"right_main"})[1].text
    print(job_salary_div)
    # print("(工作內容)=============================================================")
    job_content_div = new.find("div",{"class":"job_explain mt"}).find_all("span",{"class":"right_main"})[0].text
    print(job_content_div)
    # print("(擅長)=============================================================")
    try:
        b=h3_results.index("技能與求職專長")
        job_can_div = new.find_all("div",{"class":"job_explain"})[b+1].find_all("span",{"class":"right_main"})
        for i1 in job_can_div:
            job_can_text =i1.text  
            # print(job_can_text)
    except:
        job_can_text=" "
        # print(job_can_text)
    # print("(其他條件)===================================")
    try:
        c=h3_results.index("其他條件")
        job_other_div = new.find_all("div",{"class":"job_explain"})[c+1].find_all("span",{"class":"exception"})
        for i2 in job_other_div:
            job_other_text=i2.text
            # print(job_other_text)
    except:
        job_other_text=" "
        # print(job_other_text)
    detail_result ={
             '學歷': test4,
            '科系': test3,
            '薪資':  job_salary_div,
            '工作內容':job_content_div,
            '擅長工具':job_can_text,
            '其他條件':job_other_text
        }
    detail_results.append(detail_result)
# print(h3_results)
print(len(detail_result))
print(len(detail_results))
for i in range(min(len(detail_results),len(results2))):
    detail_results[i].update(results2[i])
print(len(results2))


labels =  ['職缺','公司名稱','公司地址','學歷','科系','薪資','工作內容','擅長工具','其他條件']
dct_arr = detail_results
try:
    with open('50page_all_column.csv', 'w',encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=labels)
        writer.writeheader()
        for elem in dct_arr:
            writer.writerow(elem)
except IOError:
    print("I/O error")

In [ ]:
# test list in list
e= ["1","2","3"]
f=["1","2","3"]
g= ["1","2","3"]
k=[]
k.append(e)
k.append(f)
k.append(g)
print(k)

In [5]:
all_h3_list =[]
detail_results=[]
for new in detail_pagesource_results:         # print(type(soup2))
    test2 = new.find_all("div",{"class":"job_explain"})[1:-1] 
    h3_results=[]
    for okok in test2:
        h3_result=okok.find("h3").text
        print(h3_result)
        h3_results.append(h3_result)
print(h3_result)



工作條件
企業福利
工作條件
企業福利
技能與求職專長
徵才特色
工作條件
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
工作條件
企業福利
技能與求職專長
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
工作條件
企業福利
技能與求職專長
工作條件
企業福利
技能與求職專長
工作條件
企業福利
技能與求職專長
其他條件
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
工作條件
企業福利
工作條件
企業福利
技能與求職專長
徵才特色
工作條件
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
工作條件
企業福利
技能與求職專長
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
技能與求職專長
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件
企業福利
其他條件
徵才特色
工作條件


AttributeError: 'NoneType' object has no attribute 'text'

# (勿刪) 合併搜尋結果所需欄位+ 每頁詳細資料欄位

In [ ]:
# print(detail_results)
# print(results2)
for i in range(min(len(detail_results),len(results2))):
    detail_results[i].update(results2[i])
print(len(results2))
for i in detail_results:
    # print(len(i))
    print(i)
  

# 匯出成csv

In [ ]:
import csv
labels =  ['職缺','公司名稱','公司地址','學歷','科系','薪資','工作內容','擅長工具','其他條件']
dct_arr = detail_results
try:
    with open('twopage_all_column.csv', 'w',encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=labels)
        writer.writeheader()
        for elem in dct_arr:
            writer.writerow(elem)
except IOError:
    print("I/O error")